# Lab 4

In this lab section, you will practice implementing the forward and back-propagation of a simple neural network in numpy. A neural network can be thought of as a composition of functions where each layer represents a function. During the forward pass, the input is passed through each layer sequentially. During the backward pass, we compute the derivations with respect to the parameters in each layer through chain derivation. You will be implementing classes for a fully-connected layer and the sigmoid activation function as well as the mean squared error.

# Implement the backwards function of a linear layer class

The input to backward should be the gradient of the final loss with respect to the output of the current layer. The output of backprop should be the gradient of the final loss with respect to the input of the current layer, which is just the output of the previous layer. Compute the gradient of the linear transformation layer in the backward function. self.grad stores the gradient of the loss with respect to the current layer's parameter.

In [1]:
import numpy as np

class Linear():
    '''Linear layer. Parameter is NxM matrix L, input is matrix v of size B x N
    where B is batch size, output is vL.'''

    def __init__(self, input_dim, output_dim, name="Linear", std=1e-1):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = std * np.random.normal(size=(input_dim, output_dim))
        self.grad = None

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights)

    def backward(self, downstream_grad):
        '''downstream_grad should be NxB.'''
        if len(downstream_grad.shape) != 2:
            downstream_grad = np.reshape(
                downstream_grad, (len(downstream_grad), 1))

        # set self.grad = gradient of the loss with respect to the current layer's parameter
        self.grad = np.dot(self.input.T, downstream_grad)
        return np.dot(downstream_grad, self.weights.T)

# Implement the backwards function of a sigmoid layer class

In [5]:
class Sigmoid():
    '''Sigmoid layer.'''

    def __init__(self, name="Sigmoid"):
        self.name = name
        self.grad = None

    def forward(self, input):
        self.output = np.exp(input) / (1.0 + np.exp(input))
        return self.output

    def backward(self, downstream_grad):
        self.grad = self.output * (1 - self.output)
        return self.grad * downstream_grad

# Implement the backwards function of a mean squared errror class

We define the mean square error as follows:

$MSE(\hat y) = \frac{1}{2N}\sum_{i=1}^N(y_i - \hat y_i)^2$.

where $y$ is the label and $\hat y$ is your prediction. Compute the gradient of MSE w.r.t $\hat y$ in the backwards function.

In [22]:
class MeanSquaredError():
    '''cross entropy loss.'''
    def __init__(self, labels, name="Mean Squared Error"):
        self.name = name
        self.labels = labels

    def forward(self, input):
        '''input is BxN, output is B'''
        self.input = input
        return np.dot((self.labels - self.input).T, (self.labels - self.input))/(2*self.input.shape[0])

    def backward(self, downstream_grad):
        grad = -(self.labels - self.input) / (self.input.shape[0])
        return grad * downstream_grad

## Implementing a simple MLP.

In this section, we will develop a shallow neural network with fully-connected layers, aka Multi-Layer Perceptron (MLP) using the layers that have already been defined. Below, we initialize toy data that we will use to develop your implementation.

In [23]:
# setup
import numpy as np
import matplotlib.pyplot as plt

# Create some toy data
X = np.linspace(-1, 1, 100).reshape(-1,1)
y = 5*X + 2 + 0.5*np.random.normal()

print ('X = ', X.shape)
print('y = ', y.shape)

X =  (100, 1)
y =  (100, 1)


Complete the loss function where you have to call the backward_layers function

In [36]:
# 각 layer의 forward를 시켜줌
def forward_layers(layers, input):
    '''Forward pass on all the layers. Must be called before backwards pass.'''
    output = input
    for layer in layers:
        output = layer.forward(output)
    #assert output.size == 1, "only supports computations that output a scalar!"
    return output

class TwoLayerMLP(object):
    def __init__(self, input_size, hidden_size, label_size, std=1e-1, activation='sigmoid'):
        np.random.seed(0)
        self.input_size = input_size
        self.num_classes =  label_size

        self.params = {}
        self.models = [
                  Linear(input_size, hidden_size),
                  Sigmoid(),
                  Linear(hidden_size, label_size),
                ]

        self.activation = 'sigmoid'
        self.params['W1'] = self.models[0].weights
        self.params['W2'] = self.models[2].weights

    def loss(self, X, y=None, reg=0.0):
        # Unpack variables from the params dictionary
        W1 = self.params['W1']
        W2 = self.params['W2']
        _, C = W2.shape
        N, D = X.shape

        ###########################################################################
        # Computes the loss
        ###########################################################################
        scores = forward_layers(self.models, X)
        loss_layer  = MeanSquaredError(y)
        loss = loss_layer.forward(scores)

        grads = {}

        ###########################################################################
        # TODO: Compute the backward pass, computing the derivatives of the weights
        # and biases. Store the results in the grads dictionary. For example,
        # grads['W1'] should store the gradient on W1, and be a matrix of same size
        ###########################################################################
        downstream_grad = loss_layer.backward(1)
        
        self.backward_layers(downstream_grad)

        # call the backwards function here before filling in the dictionary values below
        grads['W2'] = self.models[-1].grad 
        grads['W1'] = self.models[0].grad

        return loss, grads

    def backward_layers(self, downstream_grad):
        '''runs a backward pass on all the layers.
        after this function is finished, look at layer.grad to find the
        gradient with respect to that layer's parameter.'''
        for layer in reversed(self.models):
            downstream_grad = layer.backward(downstream_grad)

    def train(self, X, y, X_val, y_val,
            learning_rate=1e-3, learning_rate_decay=0.95,
            reg=1e-5, num_epochs=10,
            batch_size=1, verbose=False):

        num_train = X.shape[0]
        iterations_per_epoch = 1 #int(max(num_train / batch_size, 1))
        epoch_num = 0

        # Use SGD to optimize the parameters in self.model
        loss_history = []
        grad_magnitude_history = []
        train_acc_history = []
        val_acc_history = []

        np.random.seed(1)
        for epoch in range(num_epochs):
            # fixed permutation (within this epoch) of training data
            perm = np.random.permutation(num_train)

            # go through minibatches
            for it in range(iterations_per_epoch):
                X_batch = None
                y_batch = None

                # Create a random minibatch
                idx = perm[it*batch_size:(it+1)*batch_size]
                X_batch = X[idx, :]
                y_batch = y[idx]
                # Compute loss and gradients using the current minibatch
                loss, grads = self.loss(X_batch, y=y_batch, reg=reg)
                #print("loss", loss)
                loss_history.append(loss)

                # do gradient descent
                for param in self.params:
                    self.params[param] -= grads[param] * learning_rate

                # record gradient magnitude (Frobenius) for W1
                grad_magnitude_history.append(np.linalg.norm(grads['W1']))

            # Decay learning rate
            learning_rate *= learning_rate_decay

        return {
          'loss_history': loss_history,
          'grad_magnitude_history': grad_magnitude_history,
        }

In [37]:
input_size = 1
hidden_size = 10
num_classes = 1

net = TwoLayerMLP(input_size, hidden_size, num_classes)
scores = forward_layers(net.models, X)
print ('(1) Your scores:\n')
print (np.linalg.norm(scores))
print ('\n')
correct_norm = 2.00385
# # The difference should be very small (< 1e-4)
print ('Difference between your scores and correct scores:')
print (np.sum(np.abs(np.linalg.norm(scores) -correct_norm)))
print ('\n')

loss, _ = net.loss(X, y, reg=0.1)
correct_loss = 5

# Since we generate random data your loss would not be the same as the correct loss.
# However, the difference should fairly small (less than 1 or 2)
print ('(2) Your loss: %f'%(loss))
print ('Difference between your loss and correct loss:')
print (np.sum(np.abs(loss - correct_loss)))

(1) Your scores:

2.003850658289494


Difference between your scores and correct scores:
6.58289494026576e-07


(2) Your loss: 4.373174
Difference between your loss and correct loss:
0.6268261296629332


/var/folders/xt/y_17jz6n3p77v5p6cbm1tstr0000gn/T/ipykernel_41027/4022588834.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print ('(2) Your loss: %f'%(loss))


In [39]:
# Use numeric gradient checking to check your implementation of the backward pass.
# If your implementation is correct, the difference between the numeric and
# analytic gradients should be less than 1e-8 for each of W1, W2, b1, and b2.
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))


def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
  """
  a naive implementation of numerical gradient of f at x
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """

  fx = f(x) # evaluate function value at original point
  grad = np.zeros_like(x)
  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    oldval = x[ix]
    x[ix] = oldval + h # increment by h
    fxph = f(x) # evalute f(x + h)
    x[ix] = oldval - h
    fxmh = f(x) # evaluate f(x - h)
    x[ix] = oldval # restore

    # compute the partial derivative with centered formula
    grad[ix] = (fxph - fxmh) / (2 * h) # the slope
    if verbose:
      print (ix, grad[ix])
    it.iternext() # step to next dimension

  return grad

loss, grads = net.loss(X, y, reg=0.1)

# these should all be very small
for param_name in grads:
    f = lambda W: net.loss(X, y, reg=0.1)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print ('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

W2 max relative error: 1.010465e-10
W1 max relative error: 2.707036e-09


/var/folders/xt/y_17jz6n3p77v5p6cbm1tstr0000gn/T/ipykernel_41027/1777358646.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  grad[ix] = (fxph - fxmh) / (2 * h) # the slope
